In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

In [ ]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data/training_images'

img_width, img_height = 640., 480.

In [ ]:
train_df = pd.read_csv(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data', 'training.csv'))
im_list = train_df.image_name.tolist()

In [ ]:
print(len(im_list))
print(im_list[:5])
train_df.head()

In [ ]:
##### img_width, img_height = 640, 480
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2.h5" # Change for every new training session 

In [ ]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)

In [ ]:
# base_model.save('resnet50.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'resnet50.h5'))

In [ ]:
for layer in base_model.layers[:-5]:
    layer.trainable = False
base_model.summary()

In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.0)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
predictions = Dense(4, activation="linear")(x)

In [ ]:
model = Model(input = base_model.input, output = predictions)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.summary()

In [ ]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [ ]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

In [ ]:
model.load_weights(chkpt_dir)

In [ ]:
model.compile(loss = "mse", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['mse', mIoU])

In [ ]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [ ]:
train_df = pd.read_csv(os.path.join(base_dir, 'data', 'train.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'data', 'val.csv')))

In [ ]:
l = []
for im in val_df.image_name:
    img = cv2.imread(os.path.join(base_dir, 'data', 'validate', im))
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
y_val = val_df.iloc[:,1:].values.astype(np.float64)

y_val[:, 0] = y_val[:, 0]# / img_width
y_val[:, 1] = y_val[:, 1]# / img_width
y_val[:, 2] = y_val[:, 2]# / img_height
y_val[:, 3] = y_val[:, 3]# / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)

In [ ]:
# model.load_weights(chkpt_dir)

In [ ]:
for epoch in range(epochs):
    for i in range(13):
        l = []
        for im in train_df.image_name[i * 1000: (i + 1) * 1000]:
            img = cv2.imread(os.path.join(base_dir, 'data', 'train', im))
            l.append(np.expand_dims(img, axis = 0))
        x_train = np.vstack(l)
        y_train = train_df[i * 1000: (i + 1) * 1000].iloc[:,1:].values
        y_train[:, 0] = y_train[:, 0]# / img_width
        y_train[:, 1] = y_train[:, 1]# / img_width
        y_train[:, 2] = y_train[:, 2]# / img_height
        y_train[:, 3] = y_train[:, 3]# / img_height
        
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)

In [ ]:
model.save('transfer_resnet50-2.h5')

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)